Several functions are in the train.py
TODO: MOVE THEM TO UTILZE

In [1]:
import glob
from itertools import chain
import os
import argparse
import random
import zipfile
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from linformer import Linformer
import torchvision.transforms.functional as TF
from torch.autograd import Variable
import torch.nn.functional as F
import io
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#from vit_pytorch.efficient import ViT
#from vit_pytorch.efficient import ViT
from vit_pytorch.vit import ViT
from vit_pytorch.deepvit import DeepViT


import wandb

import sys
# import custom utilize

from util.utilize import *
import util.utilize as ut


In [6]:
a = get_dataUNI(infer_3d=True)[1].__getitem__(4)[0].cpu().numpy()
b = get_dataUNI()[1].__getitem__(10)[0].cpu().numpy()
# show img
import matplotlib.pyplot as plt
plt.imshow(a[0,:,:,200], cmap='gray')

/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the 

IndexError: index 200 is out of bounds for axis 3 with size 195

In [2]:
a = get_dataUNI(infer_3d=True)
datas = DataLoader(a[1], batch_size=1, shuffle=False)
vote(datas, is_MIX=True,save_path='/code/covid_ckpts/temp/',model = get_model_octa_resume(outsize=5, path='/code/covid_ckpts/aug_octa_split1/valacc0.7895_tpr_0.8222.pt'))


sefl= torch.Size([50, 1024]) torch.Size([1, 1024]) Dropout(p=0.2, inplace=False)


/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


TPR: 0.19047619047619047, TNR: 0.7341269841269841


/code/chen/COVID-Net/util/utilize.py:55: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.where(tp+fn == 0, 0, tp/(tp+fn))


In [ ]:
# lr: 1e-3 3e-3 1e-4 3e-4
# split 0 1 2 run name: backbone lr loss split
lrs, splits = [1e-3, 1e-4, 3e-4], [0, 1, 2]
for lr in lrs:
    for split in splits:
        run_name = f"covid_vit_lr{lr}_split{split}"
        print(run_name)
        wandb.init(project='vit', name=run_name)
        args = {
            'device': torch.device("cuda:0"),
            # 'model': get_model_octa_resume(outsize=5, path='/code/covid_ckpts/octa_split1/1527_val_acc0.694444477558136.pt', dropout=0.15), # 写一个函数获取不同的model和预训练model，这样方便些
            # 'model': get_model_conv(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt'),
            'model': get_model_oct_withpretrain(pretrain_out=2,outsize=5, path='/code/chen/pretrain/net.pt', dropout=0.15),
            # 'model': get_model_oct_withpretrain(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt', dropout=0.15),
            'save_path': f'/code/covid_ckpts/temp/', 
            'criterion': mixloss(bce_weight=0.9999),
            'epochs': 200,
            'lr': lr,
            'batch_size': 300,
            'datasets': get_dataUNI(split_idx=split),
            'is_echo': False,
            'optimizer': optim.Adam,
            'scheduler': optim.lr_scheduler.CosineAnnealingLR,
            'train_loader': None,
            'eval_loader': None,
            'shuffle': True,
            'is_MIX': True, # 是否使用mixloss input
            # 'wandb': ['visual-intelligence-laboratory','delete_vit3d'],
            'wandb': ['2065136374','exp',run_name],
            'metric_path': '/code/chen/COVID-Net/log/covid-vit.csv'  , #所有实验统一保存结果的csv
        }
        ut.device = args['device']
        # do not change the excution 
        torch.autograd.set_detect_anomaly(True)
        train_epoch(**args)

covid_vit_lr0.001_split0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 2065136374. Use `wandb login --relogin` to force relogin


sefl= torch.Size([50, 1024]) torch.Size([1, 1024]) Dropout(p=0.15, inplace=False)
Train dataset length: 1050, Val dataset length: 19


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


  0%|          | 0/4 [00:00<?, ?it/s]

...

TPR: 0.29243827160493824, TNR: 0.6773765432098765


/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/code/chen/COVID-Net/util/utilize.py:55: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.where(tp+fn == 0, 0, tp/(tp+fn))
/code/chen/COVID-Net/util/utilize.py:778: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futu

TPR: 0.24814814814814815, TNR: 0.6204678362573098


KeyError: 'val_acc'

: 

In [ ]:
#
args = {
    'device': torch.device("cuda:2"),
    # 'model': get_model_octa_resume(outsize=5, path='/code/covid_ckpts/octa_split1/1527_val_acc0.694444477558136.pt', dropout=0.15), # 写一个函数获取不同的model和预训练model，这样方便些
    # 'model': get_model_conv(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt'),
    'model': get_model_oct_withpretrain(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt', dropout=0.15),
    'save_path': '/code/covid_ckpts/octa_split1/', 
    'criterion': mixloss(bce_weight=1),
    'epochs': 2000,
    'lr': 1e-4,
    'batch_size': 300,
    'datasets': get_dataUNI(),
    'is_echo': False,
    'optimizer': optim.Adam,
    'scheduler': optim.lr_scheduler.CosineAnnealingLR,
    'train_loader': None,
    'eval_loader': None,
    'shuffle': True,
    'is_MIX': True, # 是否使用mixloss input
    # 'wandb': ['visual-intelligence-laboratory','delete_vit3d'],
    'wandb': ['2065136374','delete']
}
ut.device = args['device']
# do not change the excution 
args['optimizer'] = args['optimizer'](args['model'].parameters(), lr=args['lr'], weight_decay=1e-3)
args['scheduler'] = args['scheduler'](args['optimizer'], T_max= args['batch_size'], eta_min=8e-6)
args['train_loader'], args['eval_loader'] = DataLoader(args['datasets'][0], args['batch_size'], args['shuffle']), DataLoader(args['datasets'][1], args['batch_size'], args['shuffle'])
torch.autograd.set_detect_anomaly(True)
train_epoch(**args)

NameError: name 'torch' is not defined